<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

### Dekorateure

Der Python-Interpreter besitzt eine maximale Rekursionstiefe.
Beim Erreichen des festgesetzten Limits bricht der Interpreter die Rekursion ab und wirft eine `RecursionError` Exception.
Wie hoch das eingestellte Limit ist, können Sie über das `sys`-Modul einsehen:

In [ ]:
import sys
sys.getrecursionlimit()

Die folgende Funktion beinhaltete eine theoretisch endlose Rekursion ab.
Nach einem Aufruf sollten Sie den *Call Stack* mit der entsprechenden Fehlermeldung sehen.

In [ ]:
counter = 0
def f():
    global counter
    counter+=1
    return f()

f()

In [ ]:
counter

**Aufgabe:** Schreiben Sie einen *Function Decorator* `safe_recursion` mit dem Sie eine Rekursive Funktion absichern können.
Die dekorierte Funktion soll die Rekursion in einem `try`-`except`-Block aufrufen und eine `RecursionError` Exception abfangen.
Der Decorator soll selbsständig einen Counter implementieren mit der die Rekursionstiefe der Funktion mitverfolgt wird. 
Im Fall eines `RecursionError` soll die Rekursionstiefe ausgegeben werden.

*Hinweis:* Um den Zähler in der Decorator Funktion zu speichern, können die dem Funktionsobjekt ein Attribut hinzufügen. Das sieht etwas "sonderbar" aus, da Funktionen in Python aber *ganz normale Objekte* sind, ist dies ohne weiteres möglich.
In der folgenden Funktion `test` weisen wir innerhalb des Funktionskörpers dem Attribut `test.wert`der Wert 42 zu.
`wert` ist nun ein Attribut des Funktionsobjekts `test`.
Wir können den Wert also mit `test.wert` abrufen.

In [ ]:
def test():
    test.wert = 42
    return 123

print(test())
test.wert

In [ ]:
def safe_recursion(func):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
@safe_recursion
def f():
    return f()
f()

### SQLite Datenbank

In dieser Aufgabe wollen wir eine SQLite Datenbank verwenden, um Musik Alben lokal abzuspeichern.
Um an sinnvolle Daten zu kommen, verwenden wir die frei zugängliche Datenbank des [Musicbrainz Projekts](https://musicbrainz.org/).

In [ ]:
import sqlite3
import musicbrainzngs as mb

mb.set_useragent("Skriptsprachen","0.1", contact="none")

def alben_von(artist):
    #q = mb.search_releases(artist=artist, type="Album")
    search_results = mb.search_artists("Neil Young")
    artist_id = search_results['artist-list'][0]['id']
    q = mb.browse_releases(artist=artist_id, release_type=['album'], limit=100)
    result = []
    for i,r in enumerate(q['release-list']):
        if 'date' in r and 'title' in r:
            result.append((artist, r['title'], r['date']))
    return result
        
alben_von("Neil Young")[0:9]
    

Nun verbinden wir uns mit einer lokalen SQLite Datenbank, in der wir die heruntergeladenen Informationen ablegen wollen.
Wenn Sie eine Verbindung zu einer nicht vorhandenen SQLite-Datenbankdatei herstellen, erstellt SQLite automatisch die neue Datenbank mit dem abgegebenen Dateinamen.

Eine Aufgebaute DB-Verbindung sollte in jedem Fall wieder geschlossen werden, da ansonsten die Datenbank gelockt bleibt.
Wir werden gleich eine Möglichkeit sehen, wie man das explizite Schließen umgehen kann.

In [ ]:
con = sqlite3.connect('db.sqlite3', timeout=10)
con.close()

Über Context-Manager, die mit dem Schlüsselwort `with` erzeugt werden, kann man den Zugriff auf geteilte Ressourcen, wie Dateien oder eben Datenbanken kontrollieren.
Ein Context-Manager Klasse implementiert die *Magic Methods* `__enter__` und `__exit__`.
So kann die die Ressource in der `__enter__`-Methode angefordert und beim Verlassen des `with` Blockes wieder über die `__exit__`-Methode freigegeben werden. 
 

In [ ]:
with sqlite3.connect('db.sqlite3', timeout=10) as con:
    print("Verbunden mit db.sqlite3")

Da wir in dieser Aufgabe immer mit einer leeren Datenbank starten möchten, löschen wir zunächst die einige *Table* unserer DB.
Dabei gibt es das Problem, dass die Tabelle beim ersten Aufruf noch gar nicht vorhanden ist:

In [ ]:
with sqlite3.connect('db.sqlite3', timeout=10) as con:
    con.execute("DROP TABLE Albums")

**Aufgabe:** Sichern Sie die `DROP TABLE` Anfrage mit Ausnahmebehandlung ab, sodass es nicht mehr zu einem Fehler kommt, sondern nur noch eine Hinweis ausgegeben wird, dass die Tabelle nicht existiert.
Um welche Exception-Klasse handelt es sich? Fangen Sie nur den speziellen Fehler ab, der bei dem Datenbankzugriff Auftritt. Geben Sie die ursprüngliche Fehlermeldung der Exception ("no such table: Albums") mit aus.

In [ ]:
with sqlite3.connect('db.sqlite3', timeout=10) as con:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
%rerun

Nun erstellen wir eine neue *Table* in der Datenbank.

In [ ]:
with sqlite3.connect('db.sqlite3', timeout=10) as con:
    cur = con.cursor()
    albums_sql = """
    CREATE TABLE Albums (
        id integer PRIMARY KEY,
        artist text NOT NULL,
        title text NOT NULL,
        date date,
        CONSTRAINT unq UNIQUE (artist, title))"""
    cur.execute(albums_sql)


Wir können nun die Veröffentlichungen von Musikkünstlern oder Bands herunterladen und in der Tabelle ablegen.
Allerdings liefert uns Musicbrainz häufig auch Varianten ein und desselben Albums.
Wir möchten aber einen Albumtitel nur einmalig in unserer DB ablegen.
Daher auch die Forderung `UNIQUE (artist, title)` im Datenbankschema.
Wenn wir versuchen, ein gleichnamiges Album erneut einzufügen, resultiert dies in einem Fehler.

In [ ]:
a = alben_von("Neil Young")

for r in a:
    insert_sql = f"INSERT INTO Albums (artist, title, date) VALUES (\"{r[0]}\", \"{r[1]}\", \"{r[2]}\")"
    with sqlite3.connect('db.sqlite3', timeout=10) as con:
        cur.execute(insert_sql)

**Aufgabe:** Fangen Sie den Fehler beim Eintragen doppelter Titel ab, sodass nur noch ein Hinweis auf der Standardausgabe angezeigt wird.

In [ ]:
a = alben_von("Neil Young")

for r in a:
    insert_sql = f"INSERT INTO Albums (artist, title, date) VALUES (\"{r[0]}\", \"{r[1]}\", \"{r[2]}\")"
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
%rerun

Wir können nun noch den Inhalt der Datenbank ausgeben:

In [ ]:
with sqlite3.connect('db.sqlite3', timeout=10) as con:
    cur.execute("SELECT * FROM Albums")
    print(cur.fetchall())

Wir definieren nun eine Funktion, mit der wir ein Release-Datum aus der Datenbank lesen können:

In [ ]:
def get_date_of_release(artist, title):
    with sqlite3.connect('db.sqlite3', timeout=10) as con:
        get_sql = f"SELECT date FROM Albums WHERE artist='{artist}' AND title='{title}'"
        cur.execute(get_sql)
        r = cur.fetchone()
        if r:
            return r[0]
        else:
            return None

Da man annehmen kann, dass sich die Release Daten nicht ändern, können wir die Rückgaben der Methode lokal cachen.
Damit spart man sich Zugriffe auf die Datenbank, für wiederkehrende Anfragen mit gleichen Parametern.

**Aufgabe:** Verwenden Sie den Caching Decorator (aus dem Arbeitsblatt 15) für die Methode `get_date_of_release`.

In [ ]:
def cached(f):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
@cached
def get_date_of_release(artist, title):
    with sqlite3.connect('db.sqlite3', timeout=10) as con:
        get_sql = f"SELECT date FROM Albums WHERE artist='{artist}' AND title='{title}'"
        cur.execute(get_sql)
        r = cur.fetchone()
        if r:
            return r[0]
        else:
            return None

In [ ]:
%timeit -n 1 -r 1 d = get_date_of_release("Neil Young", "Freedom")

In [ ]:
%timeit -n 1 -r 1 d = get_date_of_release("Neil Young", "Freedom")